In [22]:
# imports
import pandas as pd
import json
import os
from IPython.core.debugger import set_trace
# change directory
# os.chdir(os.path.dirname(os.getcwd()))

In [23]:
os.chdir('c:\\Users\\sonny\\Documents\\Data Science\\TechLabs Project\\src')

In [24]:
# settings for Lottie (Animations)

data_points_url = 'https://assets5.lottiefiles.com/packages/lf20_6dboqita.json'
wondering_url = 'https://assets4.lottiefiles.com/packages/lf20_pcoatxlk.json'
typing_url = 'https://assets1.lottiefiles.com/temporary_files/r5WAZZ.json'
reportlist_url='https://assets9.lottiefiles.com/packages/lf20_dq6whaxi.json'
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))

In [25]:
# set application title
application_name = 'BraSolar' 
report_status = ('Status updates will be displayed after starting the analysis.\nAnalysis at 18% - weather data loaded.\nAnalysis at 23%')

In [26]:
# import libraries & components

# import custom components
# in the future the source will need to be changed (add d07_visualisation in front of 'components')
from components import start_button, no_fig, your_fig

# 
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc 
from dash_extensions import Lottie
import pandas as pd
import os
from dash import no_update
from dash.exceptions import PreventUpdate
from time import sleep, perf_counter
import geopandas as gpd


In [27]:
# custom cardbody for status report
status_card = dbc.CardBody([
    html.H4('Status report'),
    html.P('Status updates will be displayed after starting the analysis.'),
    html.P('Analysis at 18% - weather data loaded.')

],id='status_card',style={'font_family':'Courier'})

In [28]:
# custom html.P wrapper to include status reports
status_p = html.P(['Status updates will be displayed after starting the analysis.'])

In [29]:
status_p.children = status_p.children+[html.Br(),'lololo']

In [30]:
status_p

P(['Status updates will be displayed after starting the analysis.', Br(None), 'lololo'])

In [31]:
# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.title = application_name

app.layout = dbc.Container([
    # row one
    dbc.Row([
        # row one col one
        dbc.Col([
            # logo
            dbc.Card([
                dbc.CardImg(src='../references/solarlogo.png')
            ],className='mb-2'),
            # name and link to website
            dbc.Card([
                dbc.CardBody([
                    dbc.CardLink('Sönke Maibach',target='_blank',  # _blank opens a new tab on click
                    href='https://mabikono.de')
                ])
            ],style={'height':'16vh'}),
        ], width=2),
        # row one col two
        dbc.Col([
            # title and short info
            dbc.Card([
                dbc.CardBody([
                    html.H4('{}'.format(application_name)),
                    html.P('Find out if your chosen locations are suitable for solar power.')
                ])
            ], color = 'info',style={'height':'18vh'}),
        ], width=6),
        # row one col three
        # Dark/Light Mode and Language Dropdown
        dbc.Col([
            dbc.Card([
                dbc.Button('Turn on light',color='primary', id='theme_switch')
            ])
        ],width=2)
    ], className='m-2 mt-4 h-25'),
    # row two
    dbc.Row([
        # row two col one
        # description
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(options=options,url=wondering_url, width='40%', height='40%')),
                dbc.CardBody([
                    html.H4('What is going on here?',className='card-title'),
                    html.P(
                        'Computing energy cost savings and ROI by installing '
                        'a solar system. Uses weather and topographical data '
                        'combined with personal consumption profile to find '
                        'location, tilt and alignment with maximum energy output, '
                        'energy cost savings or ROI.'
                    )
                ])
            ]),
        ], width=4),
        # row two col two
        # inputs and start button
        dbc.Col([
            dbc.Card([
                # card header for the inputs
                dbc.CardHeader(
                    Lottie(options=options,url=typing_url, height='25%', width='25%')
                ),
                dbc.CardBody([
                    html.H4('Tell me a bit about yourself'),
                    # input field for location
                    html.I("Choose a city, region or country."),
                    html.Br(),
                    dcc.Input(id="location_input", type="search",
                    value="Sachsen",
                    style={'marginRight':'20px'}),
                    html.Br(), html.Br(),

                    
                    
                    # radioitems resolution
                    html.I("Choose the resolution. A higher resolution means more data points (DP) will be analyzed."),
                    html.Br(),
                    dcc.RadioItems(id='resolution_radio',
                        options=[
                            {'label':' Fastest (~100 DP)','value':200},
                            {'label':' Balanced (~400 DP)','value':800},
                            {'label':' Very detailed (~1000 DP)','value':2000},
                        ],
                        value=200,
                        labelStyle={'display': 'block'}
                    ),

                    # Household Size input
                    html.I('How many people live in your household? Children count as 0.5.'),
                    html.Br(),
                    dcc.Input(id="household_input", type="number",
                    value=3,
                    style={'marginRight':'20px'}),
                    html.Br(), html.Br(),

                    # start button
                    html.Div('',id='old_request'),
                    start_button,

                    # choose output to plot
                    html.I('Choose what you want to see.'), html.Br(),
                    dcc.RadioItems(
                        options=[
                            {'label':'Maximum electricity output','value':'max_out'},
                            {'label':'Maximum energy cost savings','value':'max_savings'},
                            {'label':'Maximum ROI','value':'max_roi'},
                        ],
                        value='max_roi',
                        labelStyle={'display':'block'},
                        id='to_plot_radio'
                    )

                ])
            ]),
        ], width=4),
        # row two col three
        # status updates
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(Lottie(url=reportlist_url,options=options,height='20%',width='20%')),
                dbc.CardBody([
                    html.H4('Status report'),
                    html.Div('Status updates will be displayed after starting the analysis.',id='status_div'),
                    html.Div('',id='started_status'),
                    html.Div('',id='loc_ref_status'),
                    html.Div('',id='get_weather_status'),
                    html.Div('',id='loc_ref_block_status'),
                    
                    html.Span(id='status_span',children='If you see this message, function display_status is probably broken')
                ],id='status_card')
            ]),
        ], width=4),
    ],className='m-2 h-15'),
    # row three 
    dbc.Row([
        # row three col one
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='map_graph',figure={}),
                ])
            ]),
        ], width=6,style={}),
        # row three col two
        # additional graph next to map
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='histogram_graph',figure={}),
                ])
            ]),
        ], width=6),
    ],className='m-2',style={}),

    dbc.Row([
        # hidden data storage
        # status store will be used to check the last and next step in the analysis
        dcc.Store(id='status_store',data=['pre']),
        # variables to check if single steps are done
        dcc.Store(id='location_reference_store'),

        # weather_df_store keeps the current weather df, does what weather_df does in normal program
        dcc.Store(id='weather_df_store'),

        # stores for id management, could easily be improved to only use a single store
        # single store would make accessing and manipulating the data easier
        dcc.Store(id='ids_all_store'), # stores all ids with block status
        dcc.Store(id='ids_allowed_store'), # stores allowed ids
        dcc.Store(id='ids_blocked_store'), # stores blocked ids
        
        # check if weather and loc reference are available, create and save them if needed
        dcc.Store(id='weather_loaded_store'),
        # get topo data, compute slope
        dcc.Store(id='topo_loaded_store'),
        # get tilt and alignment suggestions
        dcc.Store(id='tiltalign_store'),
        # compute sun geometry, including angle of incidence
        dcc.Store(id='sungeo_store'),
        # compute loss by soiling, snow, temp and all combined
        dcc.Store(id='losses_store'),
        # get setup for maximum output and optimal personal use
        dcc.Store(id='setup_store'),
        # get financial data for all setups
        dcc.Store(id='financial_store',data=['finstart']),
        # prepare visualisation, polygons will be stored here
        dcc.Store(id='visual_store'),
        dcc.Store(id='analysis_done_store',data=[False]),

        dcc.Store(id='started_store',data=[False]),
        dcc.Store(id='step_done_store'),
        dcc.Store(id='max_out_store'),
        dcc.Store(id='smallest_area_store'),
        dcc.Store(id='user_input_store'),
        dcc.Store(id='folder_path_store')
    ],className='m-2 mb-4',style={}),
    

], fluid=True, style={'height':'100vh'})

Callbacks

In [32]:
# utility imports
import os
from time import sleep, perf_counter
from ast import literal_eval

In [33]:
# reset outputs when changing user input
# check if it's a completely new request and inform the user if that's the case
@app.callback(
    Output('start_button','n_clicks'),
    Output('status_div','children'),
    Output('old_request','children'),
    Input('location_input','value'),
    Input('resolution_radio','value'),
    Input('household_input','value')
)
def reset_analysis(loc,res,hh):
    print('clicks reset')

    # check if the results of the request are available in the database 
    # get the process id
    process_id = str(loc).lower()+'_'+str(res)
    # get the save paths
    max_out_path = '../data/03_processed/maxout_{}.pqt'.format(process_id)
    max_roi_path = '../data/03_processed/maxout_{}.pqt'.format(process_id)
    polygons_path = '../references/polygons_{}.json'.format(process_id)

    if os.path.exists(max_out_path) and os.path.exists(max_roi_path) and os.path.exists(polygons_path):
        user_info = 'This was requested before. Output should be available quickly.'
    else:
        user_info = 'This is a new request. Please be patient.'

    return 0,('Status updates will be displayed after starting the analysis.'),user_info

In [34]:
# this code is outdated, kept only for reference

# update the visible status report based on the status_store
@app.callback(
    Output('status_span','children'),
    Input('status_store','data')
)
def display_status(status):
    print('status updated')
    print(type(status))
    print(status)
    return status[0]

In [35]:
# start button func
# triggered by clicking the start button
# outputs text to the 'started_status' wrapper

@app.callback(
    Output('started_status','children'),
    Input('start_button','n_clicks')

)
def start_analysis(clicks):
    if clicks == 0:
        return ''
    else:
        return '0% done - Analysis started'

In [36]:
# when the started_status updates, make sure a location reference polygon is available
# after checking (or requesting) update the corresponding status div

# imports
from d01_data.get_address_data import load_save_reference as loadsaveref

@app.callback(
    Output('loc_ref_status','children'),
    Input('started_status','children'),
    State('location_input','value')
)
def check_locref(started,loc):
    if started == '':
        print('locref triggered but no update')
        return ''
    else:
        starttime = perf_counter()
        locref = loadsaveref(loc)
        print('locref triggered and work done')
        elapsed = perf_counter()-starttime
        return ('1% done - Location understood in '+f" {elapsed:0.2f} seconds.")

In [37]:
# after checking the location reference, check if weather data is available
# if it's missing, request it

# imports
from d01_data.get_weather_data import getload_weather

@app.callback(
    Output('get_weather_status','children'),
    Input('loc_ref_status','children'),
    State('location_input','value'),
    State('resolution_radio','value')
)
def get_weather(locref,loc,res):
    if locref == '':
        print('get weather triggered, return empty string')
        return ''
    else:
        starttime = perf_counter()
        process_id = '{}_{}'.format(str(loc).lower(),str(res))
        weather_df = getload_weather(process_id,location=str(loc),resolution=res)
        print('weather checked, id: {}'.format(process_id))
        elapsed = perf_counter() - starttime
        return ('30% done - Weather data loaded in '+f'{elapsed:0.2f} seconds')

In [38]:
# start the id management process
# create and save an id_management_df with info about allowed and disallowed ids + reasons for blocking


# imports
from d00_utils.id_management import id_manage_df, id_allow_df, update_id_block_df, updateblocks_idmanage_df
import shapely.geometry as sg

@app.callback(
    Output('loc_ref_block_status','children'),
    Input('get_weather_status','children'), # this should trigger when the weather status changes,
    State('location_input','value'),
    State('resolution_radio','value')
)
def loc_ref_block(prev_status,loc,res):
    if prev_status =='':
        print('loc ref block triggered, nothing returned')
        return ''
    else:
        starttime = perf_counter()

        # recreate the process id
        process_id = '{}_{}'.format(str(loc).lower(),str(res))
        # load the weather df, but only id col
        ids_df = pd.read_parquet('../data/01_raw/weather_{}.pqt'.format(process_id))['id_tuple'].unique()
        ids_df = pd.DataFrame(ids_df)
        print('ids df is a '+str(type(ids_df)))

        # create dfs to manage ids and blocks
        idmanage_df = id_manage_df(ids_df,id_col='id_tuple')
        block_df = pd.DataFrame(columns=['id_tuple','blocked','block reason'])

        print('pre point')
        # create a geometry col

        # update the block_df: block locations outside of the requested region
        block_df = update_id_block_df(
            ids_df,block_df,
            'outside of',
            loadsaveref(str(loc).lower()),
            'outside of '+loc,
            block_col='geometry',
            insights=False)
        print('blocks updated')
        # update the main management df
        idmanage_df = updateblocks_idmanage_df(block_df, idmanage_df)
        # change id_tuple dtype to string, then save the main df
        id_manage_df.id_tuple = id_manage_df.id_tuple.astype(str)
        id_manage_df.to_parquet('../references/id_management/id_manage_df_{}.pqt'.format(process_id))

        print('blocks by locref done')
        elapsed = starttime = perf_counter()
        return ('32% done - Weather data double-checked in '+f'{elapsed:0.2f} seconds')



In [39]:
# # main coordinator function
# # should update status store and trigger on status store changes

# # imports
# from d01_data.get_address_data import load_save_reference as loadsaveref
# from d01_data.get_weather_data import getload_weather, get_solar_irradiance
# import d00_utils.id_management as idmanage
# import shapely.geometry as sg
# from ast import literal_eval


# @app.callback(
#     Output('status_store','data'),
#     Output('weather_df_store','data'),
#     Input('start_button','n_clicks'),
#     Input('status_store','modified_timestamp'),
#     State('status_store','data'),
#     State('location_input','value'),
#     State('resolution_radio','value'),
#     State('weather_df_store','data'),
# )
# def coordinator(clicks, ts,status,loc,res,weather_df):
#     if clicks==0:
#         print('clicks at zero')
#         return ['pre'], no_update
#     else:
#         process_id = str(loc).lower()+'_'+str(res)
#         # set the status to 'started' if no parts  of the analysis are com
#         if status[0] == 'pre':
#             print('return started')
#             return ['started'],no_update
#         if status[0] == 'started':
#             # calling loadsaveref makes sure that a reference polygon is available
#             # if there's no location that fits the input this will raise an error
#             # user will wait forever, error isnt shown to user
#             locref = loadsaveref(str(loc).lower())
#             # status will be updated if step is successful
#             return ['loc_ref'],no_update
#         if status[0] == 'loc_ref':
#             # time to check if weather data is available
#             weather_df = getload_weather(process_id,location=loc,resolution=res)
#             weather_df.id_tuple = weather_df.id_tuple.astype('str')
#             print('weather data requested')
#             return ['weather_got'],weather_df.to_dict(orient='list')
#         if status[0] == 'weather_got':
#             # after getting weather data, check which locations are really part of the requested location
#             # block locations outside
#             weather_df = pd.DataFrame(weather_df)
#             # make sure id_tuple col contains actual tuples
#             weather_df.id_tuple = [literal_eval(tup) for tup in weather_df.id_tuple]

#             # prepare dataframe to save all ids and block statuses
#             ids_all_df = idmanage.id_manage_df(pd.DataFrame(weather_df),id_col='id_tuple')
#             ids_allowed = idmanage.id_allow_df(ids_all_df)
#             # empty block dataframe
#             block_df = pd.DataFrame(columns=['id_tuple','blocked','block reason'])
            
#             # create a dataframe with currently allowed ids
#             loc_gdf = pd.DataFrame(ids_allowed)
#             # add a Point column with shapely Points
#             loc_gdf['geometry'] = [
#                 sg.Point(x[1],x[0])
#                 for x
#                 in loc_gdf.id_tuple
#             ]
            
            
#             # update the block_df
#             block_df = idmanage.update_id_block_df(
#                 loc_gdf,block_df,
#                 'outside of',
#                 loadsaveref(loc),
#                 'outside of '+str(loc),
#                 block_col='geometry',
#                 insights=False)
#             # update the other id management structures
#             ids_all_df = idmanage.updateblocks_idmanage_df(block_df, ids_all_df)
#             ids_allowed = idmanage.id_allow_df(ids_all_df)
            
#             # convert id_tuple dtype to str
#             block_df.id_tuple = block_df.id_tuple.astype(str)
#             ids_all_df.id_tuple = ids_all_df.id_tuple.astype(str)
#             ids_allowed = ids_allowed.astype(str)
#             # save the id management dfs temporarily
#             block_df.to_parquet('../references/id_management/block_df_{}.pqt'.format(process_id))
#             ids_all_df.to_parquet('../references/id_management/ids_all_df_{}.pqt'.format(process_id))
#             pd.DataFrame(ids_allowed).to_parquet('../references/id_management/ids_allowed_{}.pqt'.format(process_id))

#             # actually usable locs are found, pivot weather df now



#             global df
#             df = weather_df
#             print('check if df is what you want')
#             return ['topo_got'],no_update

#         print('all if statements false, return no update')
#         return no_update, no_update

In [40]:
# # callback to check if a reference polygon for the location is available. if not a polygon will be saved
# from d01_data.get_address_data import load_save_reference as loadsaveref

# @app.callback(
#     Output('location_reference_store','data'),
#     Output('status_store','data'),
#     Input('start_button','n_clicks'),
#     State('location_input','value')
# )
# def load_locref(clicks,loc):
#     print('weather triggered')
#     if clicks == 0:
#         return no_update,['pre']
#     else:
#         sleep(1)
#         locref = loadsaveref(loc)
#         return ['done'],['loc_ref']

In [41]:
# # callback to check if weather data is available. if not data will be requested and saved
# # should be triggered after the location reference has been checked

# from d01_data.get_weather_data import getload_weather

# @app.callback(
#     # output for the df
#     # Output('weather_df_store','data'),
#     # output to trigger next callback because step is done
#     # Output('weather_loaded_store','data'),
#     Output('status_store','data'),
#     Input('location_reference_store','data'),
#     # State('location_input','value'),
#     # State('resolution_radio','value')
# )
# def load_weather(locref,loc,res):
#     return ['weather loaded']        

In [42]:
# run the server
# server can also be accessed via browser (http://127.0.0.1:8050/)
app.run_server(mode='jupyterlab',debug=True,port=8050)

G:\Anaconda\envs\mapenv\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning: The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.
  func()


clicks reset
status updated
<class 'list'>
['pre']
locref triggered but no update
get weather triggered, return empty string
loc ref block triggered, nothing returned
locref triggered and work done
available weather data will be loaded. change use_available or process_id to request new data
weather checked, id: sachsen_200
ids df is a <class 'pandas.core.frame.DataFrame'>
